# Sara och Marcus Uppgift 1

# 3.1 Kontinuerlig optimering

# 3.2 TSP datainsamling och preprocessing

In [2]:
from sko.ACA import ACA_TSP
from math import sin, asin, cos, sqrt, atan2, radians
import regex as re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np


def get_data():
    # Scrapes the data 
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome('./chromedriver', options = options)

    driver.get("https://www.infoplease.com/world/geography/major-cities-latitude-longitude-and-corresponding-time-zones")
    elem = driver.find_element_by_id("A0001770")
    text = elem.text
    driver.close()
    return text

def get_cities(data: str):
    # Formats the data into a more convenient lookup table
    rows = data.split('\n')
    cities = []
    for row in rows[2:]:
        city = re.findall('[A-Za-z].*, [A-Za-z]*', row)
        latitude, longitude = re.findall('[0-9]{1,} [0-9]{1,} [SNWE]', row)
        cities.append({'city': city, 
                       'latitude': latitude, 
                       'longitude': longitude})
    return cities

def to_degrees(city):
    dms_to_deg = lambda deg, minutes, direction: (float(deg) + float(minutes)/60) * (-1 if direction in ['W', 'S'] else 1)
    
    latitude = city['latitude']
    longitude = city['longitude']

    lat = dms_to_deg(*re.split('[ ]', latitude))
    long = dms_to_deg(*re.split('[ ]', longitude))

    return radians(lat), radians(long)

def calc_d(city1, city2):
    # calculates the distance between two coordinates
    lat1, lon1 = to_degrees(city1)
    lat2, lon2 = to_degrees(city2)
    
    R = 6372.8 # km

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (sin(dlat/2)**2) + (cos(lat1) * cos(lat2) * (sin(dlon/2)**2))
    c = 2 * asin(sqrt(a)) 
    
    distance = R * c
    return distance # km

def get_distance_matrix(cities):
    # Generates a distance matrix from a list of cities with coordinates
    index_matrix = [[(i, j) for j in range(len(cities))] for i in range(len(cities))]
    distance_matrix = [[calc_d(cities[i_tup[0]], cities[i_tup[1]]) for i_tup in row] for row in index_matrix]
    return np.array([np.array(row) for row in distance_matrix])

def cal_total_distance(routine) -> float:
    num_points, = routine.shape
    return sum([distance_matrix[routine[i % num_points], routine[(i + 1) % num_points]] for i in range(num_points)])

# 3.3 Baseline 1

# 3.4 Baseline 2

# 3.5 Baseline 3

In [8]:
import multiprocessing
import time
import itertools
from sko.GA import GA_TSP


def GA(setting: list):
    max_iter = setting[0]
    prob_mut = setting[1]
    size_pop = setting[2]
    
    ga_tsp = GA_TSP(func=cal_total_distance, n_dim=120, size_pop=size_pop, max_iter=max_iter, prob_mut=prob_mut)
    best_points, best_distance = ga_tsp.run()
    return best_points, best_distance

def run(setting: list):
    max_iter = setting[0]
    prob_mut = setting[1]
    pop_size = setting[2]
    
    setting.append(pop_size)
    
    n_cores = multiprocessing.cpu_count()
    
    start_mp = time.time()
    
    pool = multiprocessing.Pool(processes = n_cores)
    ncore_settings = [setting for i in range(n_cores)]
    result = pool.map(GA, ncore_settings)
    
    end_mp = time.time()

    start_seq = time.time()

    for i in range(n_cores):
        best_x, best_y = GA(setting)

    end_seq = time.time()

    mp_time = end_mp-start_mp
    seq_time = end_seq-start_seq
    
    print(f'\nRoutine: {list(best_x)}\n\nDistance: {round(int(best_y))}km\n\nPopulation size: {pop_size}\n\nMax iterations: {max_iter} \n\nMutation probability: {prob_mut}\n')
    print(f'With multiprocessing ({n_cores} cores) finished in {round(mp_time, 2)} seconds\n')
    print(f'Without multiprocessing finished in {round(seq_time, 2)} seconds\n')
    print('-'*30)
    

In [8]:
data = get_data()
cities = get_cities(data)
distance_matrix = get_distance_matrix(cities)

In [9]:
all_settings = list(itertools.product([100, 500, 1000], [0.001, 0.01, 0.05]))
all_settings = list(map(list, all_settings))
all_settings = [setting + [40] for setting in all_settings]

for setting in all_settings:
    run(setting)


Routine: [102, 10, 29, 75, 105, 86, 68, 98, 5, 71, 82, 33, 108, 7, 72, 69, 80, 55, 92, 51, 50, 116, 14, 47, 95, 13, 9, 12, 93, 18, 24, 99, 117, 113, 26, 36, 25, 4, 107, 64, 58, 84, 115, 42, 48, 94, 20, 1, 16, 0, 57, 43, 49, 67, 3, 70, 19, 87, 37, 73, 52, 38, 109, 77, 79, 44, 22, 91, 103, 21, 61, 28, 83, 27, 96, 118, 56, 2, 85, 66, 89, 60, 90, 11, 74, 97, 45, 112, 34, 101, 32, 15, 41, 111, 59, 54, 106, 8, 104, 40, 81, 6, 78, 23, 110, 88, 46, 35, 39, 119, 31, 65, 30, 17, 100, 76, 63, 62, 53, 114]

Distance: 685906km

Population size: 40

Max iterations: 100 

Mutation probability: 0.001

With multiprocessing (8 cores) finished in 1.2 seconds

Without multiprocessing finished in 5.37 seconds

------------------------------

Routine: [112, 47, 89, 108, 51, 53, 58, 105, 97, 49, 95, 67, 99, 21, 65, 22, 100, 43, 11, 14, 15, 102, 26, 6, 66, 70, 93, 57, 98, 25, 109, 8, 111, 20, 48, 79, 2, 63, 91, 0, 29, 46, 30, 16, 62, 31, 90, 114, 18, 106, 4, 86, 83, 61, 88, 23, 41, 50, 104, 7, 35, 118, 37, 1

Process ForkPoolWorker-32:
Process ForkPoolWorker-66:
Process ForkPoolWorker-37:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-31:
Process ForkPoolWorker-7:
Process ForkPoolWorker-45:
Process ForkPoolWorker-49:
Process ForkPoolWorker-42:
Process ForkPoolWorker-63:
Process ForkPoolWorker-36:
Process ForkPoolWorker-12:
Process ForkPoolWorker-71:
Process ForkPoolWorker-67:
Process ForkPoolWorker-55:
Process ForkPoolWorker-61:
Process ForkPoolWorker-21:
Process ForkPoolWorker-13:
Process ForkPoolWorker-4:
Process ForkPoolWorker-57:
Process ForkPoolWorker-22:
Process ForkPoolWorker-34:
Process ForkPoolWorker-47:
Process ForkPoolWorker-53:
Process ForkPoolWorker-1:
Process ForkPoolWorker-70:
Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Process ForkPoolWorker-56:
Process ForkPoolWorker-59:
Process ForkPoolWorker-20:
Process ForkPoolWorker-26:
Process ForkPoolWorker-44:
Process ForkPoolWorker-69:
Process ForkPoolWorker-33:
Process 

  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessin

  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*sel

  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/process.py", line 99, in run
   

  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.

  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/

KeyboardInterrupt
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/pyt

# 3.6 Ant Colony Optimization

In [17]:
data = get_data()
cities = get_cities(data)
distance_matrix = get_distance_matrix(cities)

In [ ]:
num_points = len(cities)
size_pop = 50
max_iters = [100, 250, 500]

for max_iter in max_iters:
    start = time.time()
    aca = ACA_TSP(func=cal_total_distance, 
                  n_dim=num_points,
                  size_pop=size_pop, max_iter=max_iter,
                  distance_matrix=distance_matrix)

    best_x, best_y = aca.run()
    stop = time.time()

    time_elapsed = stop-start
    print(f'\nRoutine: {best_x}\n\nDistance: {round(best_y)}km\n\nTime: {round(time_elapsed/60, 2)} min\n\nPopulation size: {size_pop}\n\nMax iterations: {max_iter}\n')
    print('-'*20)



Routine: [  0  41  44  12  39  15  65  93  21  61  68  64  85   3  19  47  42  78
 119  75  99  83 113  14  95 115  24  13 106  23  86   4   6 112  26  72
  38  88 107 102  50  77 117  22  91  66  70   9   2  18  67  63  57  49
  45 114  74  71  32  97  48 110  84  17   8  96  82 104  10 116  80  87
  79 111  33  52  28  69  37  94  73 108  20   7 118  51   1  92  55 105
  59  27  54  58  81 109  56  29  40  60  53  25  76   5  35 101  62  16
  89  46  30  90  43  31  11 100  98 103  36  34]

Distance: 164834km

Time: 0.5 min

Population size: 50

Max iterations: 100

--------------------
